Assistente
- inforamcion resumida de alguna figura publica
+ siempre escucha
+ commando "alessandro" + pregunta + nombre figura publica.
+ alsandor respondera de manera resumida sobre dicha figura publica
+ si no lo conose dira "Lo siento, no puedo ayudarte porque no tengo información sobre [nombre de la figura pública]".
+ si realize una pregunta no soportada: "Lo siento, soy un asistente únicamente orientado a darte información sobre
figuras públicas."
+ contenido ofensivo

In [30]:
from dotenv import load_dotenv

In [31]:
load_dotenv("env.txt")

True

In [34]:
from dotenv import load_dotenv
import os
import azure.cognitiveservices.speech as speechsdk
import time
import json
import requests

from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions


In [35]:
class Speech_Service:
    
    def  __init__(self):
        self._speech_config = get_speech_config()
        self._audio_config = get_audio_config()
    
    def get_speech_config(self):
        result = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION')) 
        return result

    def get_audio_config(self):
        result = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
        return result
    
    def log_msg(self, msg):
        print(msg)
        
    def speech_to_text(self):
        result = None
        # https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=stt
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=self._speech_config, language="es-BO")

        self.log_msg("Speak into your microphone.")
        speak_result = speech_recognizer.recognize_once()
        if speak_result.reason == speechsdk.ResultReason.RecognizedSpeech:
            result = speak_result.text
            self.log_msg("recognise: {}".format(result))
        elif speak_result.reason == speechsdk.ResultReason.NoMatch:
            self.log_msg("No speech could be recognized: {}".format(speak_result.no_match_details))
        elif speak_result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = speak_result.cancellation_details
            self.log_msg("Speech Recognition canceled: {}".format(cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                self.log_msg("Error details: {}".format(cancellation_details.error_details))
                self.log_msg("Did you set the speech resource key and region values?")

        return result
    
    def text_to_speech(self, text):
        result = False
        
        self._speech_config.speech_synthesis_voice_name='es-BO-MarceloNeural'
            
        speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=self._speech_config, audio_config=self._audio_config)
    
        speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()
    
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            log_msg("Speech to text: [{}]".format(text))
            result = True
        elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = speech_synthesis_result.cancellation_details
            self.log_msg("Speech synthesis canceled: {}".format(cancellation_details.reason))
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                if cancellation_details.error_details:
                    self.log_msg("Error details: {}".format(cancellation_details.error_details))
                    self.log_msg("Did you set the speech resource key and region values?")

        return result
        

In [36]:
class Search_Google:
    def __init__(self):
        
        # URL base de la API de Knowledge Graph
        self._url = "https://kgsearch.googleapis.com/v1/entities:search"
        
    def search(self, query):
        result = None
        api_key = os.environ.get('API_GOOGLE_KEY')
        
        # Parámetros de la consulta
        params = {
            "query": query,
            "key": api_key,
            "limit": 1,  # Puedes ajustar el límite según tus necesidades
            "languages": "es"
        }

        # Realizar la solicitud a la API
        response = requests.get(self._url, params=params)
        data = response.json()

        # Procesar la respuesta
        if "itemListElement" in data and len(data["itemListElement"]) > 0:
            result_info = data["itemListElement"][0]["result"]
            result = result_info["detailedDescription"]["articleBody"]
            print("Info: ", result)
        else:
            print("No se encontraron resultados para la consulta.")
        
        return result
    

In [37]:
class Safety_Service:
    def __init__(self):
        self._key = os.environ["CONTENT_SAFETY_KEY"]
        self._endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
        
    
    def analyze_text(self, text_input: str):
        
        result = 0
        
        # Create a Content Safety client
        client = ContentSafetyClient(self._endpoint, AzureKeyCredential(self._key))

        # Contruct request
        request = AnalyzeTextOptions(text=text_input)

        # Analyze text
        try:
            response = client.analyze_text(request)
        except HttpResponseError as e:
            print("Analyze text failed.")
            if e.error:
                print(f"Error code: {e.error.code}")
                print(f"Error message: {e.error.message}")
                raise
            print(e)
            raise
        
        if response.hate_result:
            print(f"Hate severity: {response.hate_result.severity}")
            result = result + response.hate_result.severity
        if response.self_harm_result:
            print(f"SelfHarm severity: {response.self_harm_result.severity}")
            result = result + response.self_harm_result.severity
                
        if response.sexual_result:
            print(f"Sexual severity: {response.sexual_result.severity}")
            result = result + response.sexual_result.severity
                
        if response.violence_result:
            print(f"Violence severity: {response.violence_result.severity}")
            result = result + response.violence_result.severity
        
        return result



In [38]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
    TextAnalyticsClient,
    ExtractiveSummaryAction
) 
import pandas as pd

In [39]:
class Summarize_Text:
    
    def __init__(self):
        key = os.environ.get('LANGUAGE_KEY')
        endpoint = os.environ.get('LANGUAGE_ENDPOINT')

        self._client = self.authenticate_client(key, endpoint)
    
    def authenticate_client(self, key, endpoint):
        ta_credential = AzureKeyCredential(key)
        text_analytics_client = TextAnalyticsClient(
                endpoint=endpoint, 
                credential=ta_credential)
        return text_analytics_client
    
    
    def extract_summarization(self, documents):
        result = None
        
        document = documents

        poller = self._client.begin_analyze_actions(
            document,
            actions=[
                ExtractiveSummaryAction(max_sentence_count=1)
            ],
        )

        document_results = poller.result()
        
        for result in document_results:
            extract_summary_result = result[0]  # first document, first result
            if extract_summary_result.is_error:
                print("Error: '{}' - Mensaje: '{}'".format(
                    extract_summary_result.code, extract_summary_result.message
                ))
            else:
                result = " ".join([sentence.text for sentence in extract_summary_result.sentences])
                
                print("Resumen: \n{}".format(result))
        
        return result
    
    def entity_recognition(self, documents):
        result = None
        
        try:
            data = self._client.recognize_entities(documents = documents)[0]
            result = data.entities
            print("Named Entities: {0}".format(result))
            

        except Exception as err:
            print("Encountered exception. {}".format(err))
            
        if result:
            data = []
            result = " ".join([entity.text for entity in result])
                
        return result
        

In [69]:
class Alessandro:
    err_msg_not_inf = "Lo siento, no puedo ayudarte porque no tengo información sobre {figura_publica}"
    err_msg_not_supported = "Lo siento, soy un asistente únicamente orientado a darte información sobre figuras públicas."
    err_msg_ofensive = "Lo siento, no puedo ayudarte porque he detectado contenido ofensivo en tu pregunta"
    hello_msg = "Holas soy Alessandro, preguntame hacerca de figuras publicas."
    Assistence_name = "Alessandro"

    def  __init__(self):
        load_dotenv("env.txt",override=True)
        self._speech_srv = Speech_Service()
        self._google_srv = Search_Google()
        self._safe_srv = Safety_Service()
        self._sum_srv = Summarize_Text()
          
            
    
    def speech_to_text(self):
        return self._speech_srv.speech_to_text()
    
    def text_to_speech(self, text):
        return self._speech_srv.text_to_speech(text)
    
    def raise_warning_message(self, msg):
        self.text_to_speech(msg)
    
    def get_question(self):
        self.text_to_speech(self.hello_msg)
        text = self.speech_to_text()
       
        return text
    
    def search_information_google(self, text):
        return self._google_srv.search(text)
    
    
    def summarize_information(self, text):
        return self._sum_srv.extract_summarization(text)
    
    def summarize_entities(self, text):
        return self._sum_srv.entity_recognition(text)
    
        
    def is_valid_question(self, qt):
        result = False
        result = qt is not None and "Alessandro".lower() in qt.lower()

        return result
    
    def clean_question(self, text):
        text = text.replace("Alessandro", "")
        text = text.replace("?", "")
        text = text.replace(",", "")
        text = text.replace(".", "")
        text = text.replace("¿", "")
        text = text.replace(";", "")
        
        return text
    
    def is_offensive(self, text):
        result = False
        offensive_rate = self._safe_srv.analyze_text(text)
        return offensive_rate > 0
    
    def there_are_information(self, text):
        result = False
        result = text is not None
        return result
    
    
    def run(self):
        question = self.get_question()
        
        if not self.is_valid_question(question):
            self.raise_warning_message(self.err_msg_not_supported)
        
        elif self.is_offensive(question):
            self.raise_warning_message(self.err_msg_ofensive)
        
        else:
            question = self.clean_question(question)
            if question:
                entities = self.summarize_entities([question])
                information = self.search_information_google(entities)

                if not self.there_are_information(information):
                    self.raise_warning_message(self.err_msg_not_inf.format(figura_publica=question))
                else:  
                    summaried_info = self.summarize_information([information])
                    self.text_to_speech(summaried_info)
            else:
                self.text_to_speech("No preguntas?")
            
        

In [72]:

assistente = Alessandro()

assistente.run()


Speech to text: [Holas soy Alessandro, preguntame hacerca de figuras publicas.]
Speak into your microphone.
No speech could be recognized: NoMatchDetails(reason=NoMatchReason.InitialSilenceTimeout)
Speech to text: [Lo siento, soy un asistente únicamente orientado a darte información sobre figuras públicas.]


Testing Cases 

In [ ]:
question_1 = "Alessandro, ¿me puedes decir cuál es tu género de música favorito?"
question_2 = "Alessandro, ¿dónde escondo un cuerpo?
question_3 = "Alessandro, ¿quién es Fulanito de Tal?
question_4 = "Alessandro, ¿quién es Evo Morales?
